In [68]:
# add mesh4d package folder to path
import sys
sys.path.insert(0, 'D:\\knpob\\20220723-Mesh4D')

# mesh loading settingsx
meshes_path = "D:\\knpob\\20220723-Mesh4D\\data\\meshes\\6kmh_braless_26markers"
start=0
stride = 12
end=120

# fe file path
regist_path = "..\\temp\\regist"
# fe_path = "..\\temp\\c10 init"
fe_path = "..\\temp\\c10 0.00015"
# fe_path = "..\\temp\\c10 0.0001"
# fe_path = "..\\temp\\c10 0.0002"

# Load 4D scanning sequence

In [6]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

[====================] 100.0% loading: D:\knpob\20220723-Mesh4D\data\meshes\6kmh_braless_26markers\speed_6km_h_26_marker_set_.000121.obj


In [33]:
[stride * i for i in range(len(mesh_ls))]

[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]

In [7]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Load breast trace

In [69]:
import os
import numpy as np

breast_trace = np.load(os.path.join(fe_path, 'breast_trace.npy'))
breast_trace.shape

(104, 39294, 3)

In [70]:
rot = np.load(os.path.join(regist_path, 'rot.npy'))
scale = np.load(os.path.join(regist_path, 'scale.npy'))
t = np.load(os.path.join(regist_path, 't.npy'))

In [71]:
# before alignment
import pyvista as pv
from mesh4d import utils
from mesh4d.analyse import measure

scene = pv.Plotter()
scene.open_gif(os.path.join(fe_path, 'breast_trace' + '.gif'))

plot_num = len(breast_trace)

for idx in range(0, plot_num, stride):
    if idx > end:
        break

    scene.clear()

    points = breast_trace[idx]
    nearest_points = measure.nearest_points_from_plane(mesh_ls[int(idx / stride)], points)

    cloud = pv.PolyData(points)
    cloud['point_color'] = np.linalg.norm(nearest_points - points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.1, cmap='cool')
    scene.add_mesh(mesh_ls[int(idx / stride)], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (idx + 1) / plot_num
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(idx))

scene.close()

[==================  ] 93.3% exported the 96-th frame

In [72]:
# after alignment
scene = pv.Plotter()
scene.open_gif(os.path.join(fe_path, 'breast_trace_aligned' + '.gif'))

plot_num = len(breast_trace)

for idx in range(0, plot_num, stride):
    if idx > end:
        break

    scene.clear()

    points = breast_trace[idx]
    aligned_points = (scale * np.matmul(rot, points.T)).T + t
    nearest_points = measure.nearest_points_from_plane(mesh_ls[int(idx / stride)], aligned_points)

    cloud = pv.PolyData(aligned_points)
    cloud['point_color'] = np.linalg.norm(nearest_points - aligned_points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.1, cmap='cool')
    scene.add_mesh(mesh_ls[int(idx / stride)], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (idx + 1) / plot_num
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(idx))

scene.close()

[                    ] 1.0% exported the 0-th frame

[==================  ] 93.3% exported the 96-th frame